# Parsing LOGS to DF

To dos:
- [ ] Gerar um ID por linha de log (se bem que dá pra fazer isso por query, não?)
- [ ] Transformar o time em datetime para ser importado
- [ ] Separar os schemas de raw e trusted
- [ ] Escrever a documentação
- [ ] Gerar tabela de analytics e consumo
- [x] Gerar um sistema para mandar os eventos
- [ ] Montar os sistemas via DOCKERFILE


In [28]:
from sqlalchemy import create_engine
import pandas as pd
import os
from random import randrange

In [4]:
log_file = "../logs/ngix.log"

df = pd.read_csv(log_file,
              sep=r'\s(?=(?:[^"]*"[^"]*")*[^"]*$)(?![^\[]*\])',
              engine='python',
              usecols=[0, 3, 4, 5, 6, 7, 8],
              names=['ip', 'time', 'request', 'status_code', 'size', 'referer', 'user_agent'],
              na_values='-',
              header=None
                )
df.tail()

,ip,time,request,status_code,size,referer,user_agent
28982,188.64.128.39,[27/May/2015:10:05:06 +0000],"""GET /downloads/product_1 HTTP/1.1""",404,328,"""-""","""Debian APT-HTTP/1.3 (1.0.1ubuntu2)"""
28983,180.179.174.219,[27/May/2015:10:05:59 +0000],"""GET /downloads/product_2 HTTP/1.1""",404,337,"""-""","""Debian APT-HTTP/1.3 (0.9.7.9)"""
28984,180.179.174.219,[27/May/2015:10:05:11 +0000],"""GET /downloads/product_2 HTTP/1.1""",404,339,"""-""","""Debian APT-HTTP/1.3 (0.9.7.9)"""
28985,91.103.4.38,[27/May/2015:10:05:09 +0000],"""GET /downloads/product_2 HTTP/1.1""",200,951,"""-""","""urlgrabber/3.9.1 yum/3.2.29"""
28986,180.179.174.219,[27/May/2015:10:05:23 +0000],"""GET /downloads/product_2 HTTP/1.1""",200,490,"""-""","""Debian APT-HTTP/"""


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28987 entries, 0 to 28986
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ip          28987 non-null  object 
 1   time        28987 non-null  object 
 2   request     28986 non-null  object 
 3   status      28986 non-null  float64
 4   size        28986 non-null  float64
 5   referer     28986 non-null  object 
 6   user_agent  28986 non-null  object 
dtypes: float64(2), object(5)
memory usage: 1.5+ MB


# DF to POSTGRES

In [15]:
from dotenv import load_dotenv
load_dotenv('./env')

True

In [29]:
def get_engine():
    username = os.environ['POSTGRES_USER']
    password = os.environ['POSTGRES_PASSWORD']
    database = os.environ['POSTGRES_DB']
    host = os.environ['DW_HOST']
    
    engine = create_engine(f'postgresql://{username}:{password}@{host}:5432/{database}')
    return engine

In [27]:
def data_to_dw(data: pd.DataFrame, table: str):
    engine = get_engine()
    df.to_sql(table, engine, index = False, if_exists = 'replace')
    
    print("===> Success to save the data on DW.")

In [28]:
data_to_dw(df, 'raw_logs')

===> Success to save the data on DW.


# POSTGRES to ANALYTICS

In [32]:
def get_data_dw(query: str):
    engine = get_engine()
    df = pd.read_sql_query(query, con = engine)
    
    return df

In [5]:
query = 'SELECT COUNT(time) AS requests, status AS status_code FROM public.teste GROUP BY status_code ORDER BY requests DESC'
df = get_data_dw(query)
print(df.to_string(index=False))

NameError: name 'get_data_dw' is not defined

# SIMULATE LOGS INSERTION

In [218]:
def insert_log():
    df = pd.read_csv(log_file, delimiter = "\t", sep=" ", header=None)
    lines = len(df.index) + 1 # To make sure that every line will be selected
    
    rows_init = randrange(lines)
    rows_end = rows_init + randrange(11)
    new_df = df[rows_init:rows_end]
    
    new_list = new_df.values.tolist()
    
    # Write the file
    with open(log_file, "a") as file_object:
        for element in new_list:
            log = element[0]
            file_object.write(f"{log}\n")
    print("===> Writting the log.")

In [242]:
insert_log()

===> Writting the log.


# Create structure for data warehouse

In [ ]:
def structure_data_warehouse():
    # 1. 